In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [4]:
# Hyperparameters
num_epochs = 5
batch_size = 64
learning_rate = 0.001

In [5]:
# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [6]:
# Define the CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(32, 10)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.pool(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.pool(out)
        out = self.gap(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

model = CNN()


In [7]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [8]:
# Training loop
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')


Epoch [1/5], Step [100/938], Loss: 1.6049
Epoch [1/5], Step [200/938], Loss: 1.4070
Epoch [1/5], Step [300/938], Loss: 0.9846
Epoch [1/5], Step [400/938], Loss: 0.7934
Epoch [1/5], Step [500/938], Loss: 0.6497
Epoch [1/5], Step [600/938], Loss: 0.5329
Epoch [1/5], Step [700/938], Loss: 0.5529
Epoch [1/5], Step [800/938], Loss: 0.4238
Epoch [1/5], Step [900/938], Loss: 0.5033
Epoch [2/5], Step [100/938], Loss: 0.4065
Epoch [2/5], Step [200/938], Loss: 0.3004
Epoch [2/5], Step [300/938], Loss: 0.2854
Epoch [2/5], Step [400/938], Loss: 0.1963
Epoch [2/5], Step [500/938], Loss: 0.2262
Epoch [2/5], Step [600/938], Loss: 0.2717
Epoch [2/5], Step [700/938], Loss: 0.2686
Epoch [2/5], Step [800/938], Loss: 0.3108
Epoch [2/5], Step [900/938], Loss: 0.1557
Epoch [3/5], Step [100/938], Loss: 0.2425
Epoch [3/5], Step [200/938], Loss: 0.2229
Epoch [3/5], Step [300/938], Loss: 0.3079
Epoch [3/5], Step [400/938], Loss: 0.1633
Epoch [3/5], Step [500/938], Loss: 0.1240
Epoch [3/5], Step [600/938], Loss:

In [9]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the 10000 test images: {100 * correct / total:.2f}%')

Accuracy of the model on the 10000 test images: 96.39%


In [10]:
# Save the trained model
torch.save(model.state_dict(), 'mnist_model.pth')